Cargar dependencias

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


Obtenemos los datos desde el datasets

In [ ]:
# Origen: Nuestro

# Cargar datos desde un archivo CSV
# Visualiza los datos

df = pd.read_csv("datos.csv")
df.head()

Transformamos los datos de lluvia a 1 si llueve y 0 si no llueve

In [ ]:
# Origien: Del repo de clases con modificación de datos

df['llueve'] = df['precipitation_sum (mm)'].apply(lambda x: 1 if x > 0 else 0)
df.head()

Limpiamos los datos eliminando los datos nulos

In [ ]:
# Origien: Repo tutorial de pandas (Curso)
# Extra: https://www.analyticslane.com/2023/04/24/eliminar-las-filas-con-valores-nulos-en-pandas/

# Por defecto el metodo dropna() elimina las filas que tienen al menos un valor NaN
df_limpio = df.dropna()
df_limpio

Separamos los datos en datos de entrenamiento y en datos de prueba

In [ ]:
X = df_limpio[['weather_code (wmo code)', 'temperature_2m_max (°C)',
       'temperature_2m_min (°C)',
       'wind_speed_10m_max (km/h)', 'wind_gusts_10m_max (km/h)',
       'sunshine_duration (s)', 'precipitation_hours (h)']]
le = LabelEncoder()
y = df_limpio[['llueve']]
# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)



Prueba modelo


In [ ]:
# Random Forest
model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["No llueve", "Llueve"])
disp.plot(cmap='Blues')
plt.title("Matriz de Confusión: Predicción de Lluvia")
plt.show()